<a href="https://colab.research.google.com/github/greyhound101/Multihead_attention/blob/master/4_split_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
import zipfile
import datetime, os
import h5py
from tensorflow.keras.optimizers import *
import cv2
from tensorflow.keras.utils import to_categorical
import glob, os
from matplotlib import pyplot as plt
import h5py
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm
import time
import gc
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
import tensorflow as tf
import random, os, sys
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
import tensorflow as tf


In [ ]:
path = F"/content/gdrive/My Drive/check.npy" 
df=np.load(path,allow_pickle=True)
df=df.item()

In [ ]:
from tensorflow import keras
from tensorflow.keras.activations import softmax
from tensorflow.keras import backend as K
import tensorflow as tf
class LayerNormalization(Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)
    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)
    def call(self, x):
        mean = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape
def squeeze_excite_block(input_tensor, ratio=16):
    """ Create a channel-wise squeeze-excite block
    Args:
        input_tensor: input Keras tensor
        ratio: number of output filters
    Returns: a Keras tensor
    References
    -   [Squeeze and Excitation Networks](https://arxiv.org/abs/1709.01507)
    """
    init = input_tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = K.int_shape(init)[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

class abc(Layer):
    def __init__(self,inr,size,mo,up,org,**kwargs):
        super(abc, self).__init__(**kwargs)
        self.inr=inr
        self.mo=mo
        self.up=up
        self.org=org
        self.size=size
    def get_config(self):
        base_config = super(abc, self).get_config()

    def build(self, input_shape):
        super(abc, self).build(input_shape)
        self.cv1 = Conv2D(self.inr,1)
        self.cv2 = Conv2D(self.inr,1)
        
        
        
        self.dns1 = Conv2D(self.org,1,activation='relu')
        self.dns2 = Conv2D(self.org,1,activation='sigmoid')
        
        
        self.cv3 = Conv2D(1,1)
        self.up = UpSampling2D(interpolation='bilinear',size=(self.up,self.up))
        self.dns1=Dense(1)
    def call(self, img,y):
        y = self.cv1(y)
        x = self.cv2(img)
        y = self.up(y)
        
        y = Add()([y,x])
        y=GlobalAveragePooling2D()(y)
        y = Reshape((1,1,self.inr))(y)
        x = self.dns1(y)
        x = self.dns2(x)
        z = tf.math.multiply(img,x)
        
        x = ReLU()(z)
        x = K.max(x,axis=-1)
        x = Reshape((self.size,self.size,1))(x)
        
        map = softmax(x,axis=[2,3])


        return tf.math.multiply(z,map)

class abc(Layer):
    def __init__(self,inr,size,org,s,up,**kwargs):
        super(abc, self).__init__(**kwargs)
        self.inr=inr
        self.up=up
        self.org=org
        self.size=size
        self.s=s
    def get_config(self):
        base_config = super(abc, self).get_config()

    def build(self, input_shape):
        super(abc, self).build(input_shape)
        self.cv2 = Conv2D(self.inr,1)
        self.cv1 = Conv2D(self.inr,(self.s,self.s))
        
        
        self.dns1 = Conv2D(self.org,1,activation='relu')
        self.dns2 = Conv2D(self.org,1,activation='sigmoid')
        
        self.up = UpSampling2D(interpolation='bilinear',size=(self.up,self.up))
    def call(self, img,y):
        x = self.cv2(img)
        y = self.up(y)
        y = self.cv1(y)
        y = Reshape((self.size,self.size,self.inr))(y)
        x = Reshape((self.size,self.size,self.inr))(x)
        y = Add()([y,x])
        
        
        y=GlobalAveragePooling2D()(y)
        y = Reshape((1,1,self.inr))(y)
        x = self.dns1(y)
        x = self.dns2(x)
        z = tf.math.multiply(img,x)
        
        x = ReLU()(z)
        x = K.max(x,axis=-1)
        x = Reshape((self.size,self.size,1))(x)
        
        map = softmax(x,axis=[2,3])


        return tf.math.multiply(z,map)

class SpatialGate(keras.layers.Layer):
    def __init__(self,**kwargs):
        super(SpatialGate, self).__init__(**kwargs)

    def get_config(self):
        base_config = super(SpatialGate, self).get_config()

    def build(self, input_shape):
        super(SpatialGate, self).build(input_shape)
        self.cv2 = Conv2D(1,1)
    def call(self, img):
        
        img_avg = K.expand_dims(K.mean(img,-1),-1)
        img_max = K.expand_dims(K.max(img,-1),-1)
        total = Concatenate(-1)([img_avg,img_max])
        x = self.cv2(total)
        x = keras.activations.sigmoid(x)

        return tf.math.multiply(img,x)
class ChannelGate(keras.layers.Layer):
    def __init__(self,inr,ratio,**kwargs):
        super(ChannelGate, self).__init__(**kwargs)
        self.inr=inr
        self.ratio=ratio

    def get_config(self):
        base_config = super(abc, self).get_config()

    def build(self, input_shape):
        super(ChannelGate, self).build(input_shape)
        self.dns1 = Dense(self.inr/self.ratio,activation='relu')
        self.dns2 = Dense(self.inr)
        self.spt = SpatialGate()
    def call(self, img):
        
        img_avg = self.dns2(self.dns1(GlobalAveragePooling2D()(img)))
        img_max = self.dns2(self.dns1(GlobalMaxPooling2D()(img)))
        x = keras.activations.sigmoid(img_max+img_avg)
        x = Reshape((1,1,self.inr))(x)

        return self.spt(tf.math.multiply(img,x))

def Global_attention_block(inputs):
    shape=K.int_shape(inputs)
    x=MaxPooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('relu') (x)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    C_A=Multiply()([x,inputs])
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    return S_A
    
    
def Category_attention_block(inputs,classes,k):
    shape=K.int_shape(inputs)
    F=Conv2D(k*classes,1, padding='same') (inputs)
    F=BatchNormalization() (F)
    F1=Activation('relu') (F)
    
    F2=F1
    x=GlobalAveragePooling2D()(F2)
    
    x=Reshape((classes,k)) (x)
    
    x=Reshape((shape[1],shape[2],classes,k)) (F1)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    x=Activation('sigmoid') (x)
    x=Multiply()([S,x])
    M=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (x)
    M=Activation('sigmoid') (M)
    
    semantic=Multiply()([inputs,M])
    return semantic



from tensorflow import keras 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
from tensorflow import keras 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
    
    
def Category_attention_block(inputs,classes=3,k=5):
    shape=K.int_shape(inputs)
    F=Conv2D(k*classes,1, padding='same') (inputs)
    F=BatchNormalization() (F)
    F1=Activation('relu') (F)
    
    F2=F1
    x=GlobalMaxPool2D()(F2)
    
    x=Reshape((classes,k)) (x)
    S=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    
    x=Reshape((shape[1],shape[2],classes,k)) (F1)
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=False))  (x)
    x=Multiply()([S,x])
    M=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (x)
    
    semantic=Multiply()([inputs,M])
    return semantic



import pandas as pd
import imgaug as ia
from imgaug import augmenters as iaa

def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result
def Hflip( images):
		seq = iaa.Sequential([iaa.Fliplr(1.0)])
		return seq.augment_images(images)
def Vflip( images):
		seq = iaa.Sequential([iaa.Flipud(1.0)])
		return seq.augment_images(images)
def noise(images):
    ls=[]
    for i in images:
        x = np.random.normal(loc=0, scale=0.05, size=(299,299,3))
        ls.append(i+x)
    return ls
def rotate(images):
    ls=[]
    for angle in range(-15,20,5):
        for image in images:
            ls.append(rotate_image(image,angle))
    return ls
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, images, labels, batch_size=64, image_dimensions = (96 ,96 ,3), shuffle=False, augment=False):
    self.labels       = labels              # array of labels
    self.images = images        # array of image paths
    self.batch_size   = batch_size          # batch size
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(self.labels.shape[0] / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(self.labels.shape[0])

  def __getitem__(self, index):
		# selects indices of data for next batch
    indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    # select data and load images
    labels = self.labels.loc[indexes]
    img = [self.images[k].astype(np.float32) for k in indexes]
    imgH=Hflip(img)
    imgV=Vflip(img)
    imgR=rotate(img)
    images=[]
    images.extend(imgH)
    images.extend(imgV)
    images.extend(imgR)
    lbl=labels.copy()
    labels=pd.DataFrame()
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    #images = np.array([preprocess_input(img) for img in images])
    return np.asarray(images), labels.values
class DataGenerator1(tf.keras.utils.Sequence):
  def __init__(self, images, labels, batch_size=64, image_dimensions = (96 ,96 ,3), shuffle=False, augment=False):
    self.labels       = labels              # array of labels
    self.images = images        # array of image paths
    self.batch_size   = batch_size          # batch size
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(self.labels.shape[0] / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(self.labels.shape[0])

  def __getitem__(self, index):
		# selects indices of data for next batch
    indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    # select data and load images
    labels = self.labels.loc[indexes]
    img = [self.images[k].astype(np.float32) for k in indexes]
    imgH=Hflip(img)
    imgV=Vflip(img)
    imgR=rotate(img)
    images=[]
    images.extend(imgH)
    images.extend(imgV)
    images.extend(imgR)
    lbl=labels.copy()
    labels=pd.DataFrame()
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    #images = np.array([preprocess_input(img) for img in images])
    return np.asarray(images), labels.values


best_accuracy_last={}
final_accuracy_last={}
history_last={}
answers_last={}
predictions_last={}
predictions_last_best={}
times_last={}


#shuffle samples
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]



#change targets
def change(img):
    resized = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA )
    return resized




#get train and test splits
def get_trn_tst(df,tst_fold):
  idx=np.asarray(df['fold'])
  y=np.asarray(df['label'])
  y-=1
  img=np.asarray(df['image'])
  img1=[]
  for i in range(len(img)):
        img1.append(change(img[i]))
  img1=np.asarray(img1)
  del([img])
  gc.collect()
  trn_y=np.asarray(y[(idx!=tst_fold)])
  trn_img=np.asarray(img1[(idx!=tst_fold)])
  tst_y=np.asarray(y[(idx==tst_fold)])
  tst_img=img1[idx==tst_fold]
  trn_img=np.repeat(trn_img.reshape((trn_img.shape[0],224,224,1)),3,axis=3)
  tst_img=np.repeat(tst_img.reshape((tst_img.shape[0],224,224,1)),3,axis=3)
  return (trn_img.copy(),trn_y.copy()),(tst_img.copy(),tst_y.copy())

def Global_attention_block(inputs):
    shape=K.int_shape(inputs)
    p0=Lambda(lambda x: x[:,:,:, :512])(inputs)
    q0=Lambda(lambda x: x[:,:,:, 512:])(inputs)
    
    p=GlobalAveragePooling2D() (p0)
    q=GlobalAveragePooling2D() (q0)
    p=Reshape((1,1,512))(p)
    q=Reshape((1,1,512))(q)
    
    p=Conv2D(256,1, padding='same') (p)
    q=Conv2D(256,1, padding='same') (q)
    p=BatchNormalization()(p)
    q=BatchNormalization()(q)
    q=Activation('relu')(q)
    p=Activation('relu')(p)
    
    
    p=Conv2D(512,1, padding='same',activation='sigmoid') (p)
    q=Conv2D(512,1, padding='same',activation='sigmoid') (q)
    
    
    p=Multiply()([p,p0])
    q=Multiply()([q,q0])
    return Concatenate()([p,q])
from tensorflow.keras.layers import Activation,BatchNormalization
from tensorflow.keras.activations import sigmoid
def Category_attention_block(inputs):
    shape=K.int_shape(inputs)
    k=Conv2D(512,3,padding='same') (inputs)
    k=BatchNormalization()(k)
    k=Activation('relu')(k)
    k=Reshape([-1,512])(k)
    k=K.transpose()(k)#C R
    
    
    
    
    
    p1=Lambda(lambda x: x[:,:,:, :512])(inputs)
    
    pq1=Conv2D(512,3,padding='same') (p1)
    pq1=BatchNormalization()(pq1)
    pq1=Activation('relu')(pq1)
    pq1=Reshape([-1,512])(pq1)#R C
    
    pv1=Conv2D(512,3,padding='same') (p1)
    pv1=BatchNormalization()(pv1)
    pv1=Activation('relu')(pv1)
    pv1=Reshape([-1,512])(pv1)#R C
    
    q1=Lambda(lambda x: x[:,:,:, 512:])(inputs)
    qq1=Conv2D(512,3,padding='same') (q1)
    qq1=BatchNormalization()(qq1)
    qq1=Activation('relu')(qq1)
    qq1=Reshape([-1,512])(qq1)#R C
    
    qv1=Conv2D(512,3,padding='same') (q1)
    qv1=BatchNormalization()(qv1)
    qv1=Activation('relu')(qv1)
    qv1=Reshape([-1,512])(qv1)#R C
    
    map_p=K.batch_dot()([k,pq1])#C C
    map_p=sigmoid(map_p)
    pv1=K.batch_dot()([pv1,map_p])#R C
    pv1=Reshape([7,7,512])(pv1)#R C
    
    
    map_q=K.batch_dot()([k,qq1])#C C
    map_q=sigmoid(map_q)
    qv1=K.batch_dot()([qv1,map_q])#R C
    qv1=Reshape([7,7,512])(qv1)#R C
    
    return Concatenate()([pv1,qv1])

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.optimizers import *
    
def Global_attention_block(inputs):
    shape=K.int_shape(inputs)
    x=Lambda(lambda x: K.mean(x,-1))(inputs)
    x=Reshape((1,-1))(x)
    x=sigmoid(x)# 1,HW
    
    y=Reshape((-1,shape[-1]))(inputs)#HW C
    
    x=K.batch_dot(x,y)#1 C
    
    p=GlobalAveragePooling2D() (inputs)
    p=Reshape((1,1,shape[-1]))(p)#1 1 C
    p=Conv2D(shape[-1],1,activation='relu')(p)
    p=Conv2D(shape[-1],1,activation='sigmoid')(p)
    p=Reshape((1,shape[-1]))(p)#1 C
    
    
    x=Concatenate()([x,p])
    x=Reshape((1,1,2*shape[-1]))(x)#1 1 C
    x=Conv2D(shape[-1],1,activation='sigmoid')(x)
    
    return Multiply()([x,inputs])
def load_model():   
  K.clear_session() 
  mod1=DenseNet121(input_shape=(224,224,3))
  mod1.trainable=False
  out_1=mod1.layers[-3].output
  p=Lambda(lambda x: x[:,:,:, :256])(out_1)
  q=Lambda(lambda x: x[:,:,:, 256:512])(out_1)
  s=Lambda(lambda x: x[:,:,:, 512:768])(out_1)
  s=Lambda(lambda x: x[:,:,:, 768:])(out_1)
    
  p = Global_attention_block(p)
  q = Global_attention_block(q)
  r = Global_attention_block(r)
  s = Global_attention_block(s)
  
  out_1=Concatenate()([p,q,r,s])
  out_1 = GlobalMaxPooling2D()(out_1)
  out=Dense(3,activation='softmax')(out_1)
  model=Model(inputs=mod1.input,outputs=out)
  return model

In [ ]:
from matplotlib import pyplot as plt
from tensorflow.keras.applications.densenet import DenseNet121
from sklearn.metrics import accuracy_score
import gc
from tensorflow.keras.optimizers import Adam
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]



#change targets
def change(img):
    resized = cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA )
    return resized


import pandas as pd
import imgaug as ia
from imgaug import augmenters as iaa

def rotate_image(image, angle):
  image_center = tuple(np.array(image.shape[1::-1]) / 2)
  rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
  result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
  return result
def Hflip( images):
		seq = iaa.Sequential([iaa.Fliplr(1.0)])
		return seq.augment_images(images)
def Vflip( images):
		seq = iaa.Sequential([iaa.Flipud(1.0)])
		return seq.augment_images(images)
def noise(images):
    ls=[]
    for i in images:
        x = np.random.normal(loc=0, scale=0.05, size=(299,299,3))
        ls.append(i+x)
    return ls
def rotate(images):
    ls=[]
    for angle in range(-15,20,5):
        for image in images:
            ls.append(rotate_image(image,angle))
    return ls
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, images, labels, batch_size=64, image_dimensions = (96 ,96 ,3), shuffle=False, augment=False):
    self.labels       = labels              # array of labels
    self.images = images        # array of image paths
    self.batch_size   = batch_size          # batch size
    self.on_epoch_end()

  def __len__(self):
    return int(np.floor(self.labels.shape[0] / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(self.labels.shape[0])

  def __getitem__(self, index):
		# selects indices of data for next batch
    indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    # select data and load images
    labels = self.labels.loc[indexes]
    img = [self.images[k].astype(np.float32) for k in indexes]
    imgH=Hflip(img)
    imgV=Vflip(img)
    imgR=rotate(img)
    images=[]
    images.extend(imgH)
    images.extend(imgV)
    images.extend(imgR)
    lbl=labels.copy()
    labels=pd.DataFrame()
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    labels=pd.concat([labels,lbl],0)
    #images = np.array([preprocess_input(img) for img in images])
    return np.asarray(images), np.asarray(labels.values)

In [ ]:
  best_accuracy_last={}
  final_accuracy_last={}
  history_last={}
  answers_last={}
  predictions_last={}
  predictions_last_best={}
  times_last={}


In [ ]:
  def upd(dk,data):
    if dk==0:
        dk=data
    else:
        for ky in data.keys():
            dk[ky].extend(data[ky])
    return dk
  index=1
  epoch=50
  pre_acc=0
  best=0
  fold='fold_'+str(index)
  trn,tst=get_trn_tst(df,index)
  history_last[fold]=0



  plt.imshow(trn[0][0])
  plt.show()
  plt.imshow(tst[0][0])
  plt.show()



  trn_x,trn_y=unison_shuffled_copies(trn[0],trn[1])
  tst_x,tst_y=unison_shuffled_copies(tst[0],tst[1])



  model=load_model()


  
  #compiling the model
  train_data = DataGenerator(trn_x,pd.get_dummies(trn_y), batch_size=4, augment=True)
  ln=len(trn_y)
  # del([trn_x,trn_y,trn,tst])
  # gc.collect()
  #fitting the model
  #timing
  start=time.time()
  print('training')
  model.compile(optimizer=Adam(1e-2,decay=1e-3), 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
  hist=model.fit_generator(train_data,epochs=1,verbose=1,steps_per_epoch=ln//4)

  for i in model.layers:
        i.trainable=True
  model.compile(optimizer=Adam(2e-4,decay=1e-3), 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
  hist=model.fit_generator(train_data,epochs=50,verbose=1,steps_per_epoch=ln//4)
  history_last[fold]=upd(history_last[fold],hist.history)
  del([train_data,trn_x,trn_y,trn,df])
  gc.collect()
  end=time.time()
  times_last[fold]=end-start

  #getting the prediction 
  pre=model.predict(tst_x)
  



  #select the maximum position
  pre=np.argmax(pre,1)
  predictions_last[fold]=pre

  
  
  
  #getting the accuracy
  new_acc=accuracy_score(pre,tst_y)

  


  #storing the predictions
  final_accuracy_last[fold]=new_acc








  #storing the answers
  answers_last[fold]=tst_y
    
    
    
    
  #freeing memory
  del([tst_y,tst_x])
  gc.collect()

In [ ]:
plt.plot(history_last[fold]['loss'])

In [ ]:
new_acc

In [ ]:
  print(new_acc)
  index=str(index)
  type='gated'
  model1='squeezenet_'
  path='/content/gdrive/My Drive/'
  index=str(index)
  np.save(path+"/best_accuracy_all_fold_"+index+"_"+model1+"_"+type+".npy",best_accuracy_last)
  np.save(path+'/final_accuracy_all_fold'+index+"_"+model1+"_"+type+".npy",final_accuracy_last)
  np.save(path+'/history_all_fold_'+index+"_"+model1+"_"+type+".npy",history_last)
  np.save(path+'/answers_all_fold_'+index+"_"+model1+"_"+type+".npy",answers_last)
  np.save(path+'/predictions_all_fold_'+index+"_"+model1+"_"+type+".npy",predictions_last)
  np.save(path+'/predictions_all_best_fold_'+index+"_"+model1+"_"+type+".npy",predictions_last_best)
  np.save(path+'/times_all_fold_'+index+"_"+model1+"_"+type+".npy",times_last)
  model.save_weights(path+type+index+'.hdf5')